In [4]:
import os ,json 
import pandas as pd 
import traceback

In [5]:
from langchain.chat_models import ChatOpenAI 
from langchain.llms import OpenAI 
from langchain.prompts import PromptTemplate 
from langchain.chains import LLMChain 
from langchain.chains import SequentialChain 
from langchain.callbacks import get_openai_callback
import PyPDF2

In [6]:
# ! You can import environment key by 
# 1. create virtual environament variable using python-dotenv
# 2. import from terminal 
# 3. create .env file and import it
key = os.getenv("OPENAI_API_KEY")
model = "gpt-3.5-turbo"
print(key)

sk-proj-bLNLU9TklbqVfzXRt3jNT3BlbkFJ2gHWQxaQId0DehJbdYsW


In [7]:
llm  = ChatOpenAI(openai_api_key = key ,model =model ,temperature=0.5)

c:\Users\PRAMOD KHAVARE\MCQGEN\env\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [8]:
#Input Promt Skeletone will used in input prompt
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [9]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [10]:
print(TEMPLATE)


Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}




In [11]:
#This prompt will used in Input
quiz_generation_prompt = PromptTemplate(
    input_variables=['text' ,'number' ,'subject' ,'tone' ,'response_json'] ,
    template = TEMPLATE
)
quiz_generation_prompt

PromptTemplate(input_variables=['number', 'response_json', 'subject', 'text', 'tone'], template='\nText:{text}\nYou are an expert MCQ maker. Given the above text, it is your job to create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. \nMake sure the questions are not repeated and check all the questions to be conforming the text as well.\nMake sure to format your response like  RESPONSE_JSON below  and use it as a guide. Ensure to make {number} MCQs\n### RESPONSE_JSON\n{response_json}\n\n')

In [12]:
quiz_chain = LLMChain(llm= llm ,prompt= quiz_generation_prompt ,output_key='quiz' ,verbose=True)
quiz_chain

c:\Users\PRAMOD KHAVARE\MCQGEN\env\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['number', 'response_json', 'subject', 'text', 'tone'], template='\nText:{text}\nYou are an expert MCQ maker. Given the above text, it is your job to create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. \nMake sure the questions are not repeated and check all the questions to be conforming the text as well.\nMake sure to format your response like  RESPONSE_JSON below  and use it as a guide. Ensure to make {number} MCQs\n### RESPONSE_JSON\n{response_json}\n\n'), llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001F48350BC70>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001F483535A80>, temperature=0.5, openai_api_key='sk-proj-bLNLU9TklbqVfzXRt3jNT3BlbkFJ2gHWQxaQId0DehJbdYsW', openai_proxy=''), output_key='quiz')

In [13]:
#Output Template
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [14]:
#This prompt will used in Output
quiz_evzluation_prompt = PromptTemplate(
    input_variables=['subject' ,'quiz'] ,
    template = TEMPLATE2
)
quiz_evzluation_prompt

PromptTemplate(input_variables=['quiz', 'subject'], template='\nYou are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. \nif the quiz is not at per with the cognitive and analytical abilities of the students,update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities\nQuiz_MCQs:\n{quiz}\n\nCheck from an expert English Writer of the above quiz:\n')

In [15]:
review_chain = LLMChain(llm=llm ,prompt= quiz_evzluation_prompt ,output_key='review' ,verbose=True)

In [16]:
#Connnect Both Chain(input and output)
generate_evaluation_chain = SequentialChain(
    chains=[quiz_chain , review_chain ] ,input_variables=['text' ,'number' ,'subject' ,'tone' ,'response_json'],
    output_variables=["quiz", "review"], verbose=True,) 
generate_evaluation_chain

SequentialChain(verbose=True, chains=[LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['number', 'response_json', 'subject', 'text', 'tone'], template='\nText:{text}\nYou are an expert MCQ maker. Given the above text, it is your job to create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. \nMake sure the questions are not repeated and check all the questions to be conforming the text as well.\nMake sure to format your response like  RESPONSE_JSON below  and use it as a guide. Ensure to make {number} MCQs\n### RESPONSE_JSON\n{response_json}\n\n'), llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001F48350BC70>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001F483535A80>, temperature=0.5, openai_api_key='sk-proj-bLNLU9TklbqVfzXRt3jNT3BlbkFJ2gHWQxaQId0DehJbdYsW', openai_proxy=''), output_key='quiz'), LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['quiz', 

In [17]:
with open('C:\\Users\\PRAMOD KHAVARE\\MCQGEN\\data.txt' ,'r') as f:
    TEXT= f.read()
TEXT

'Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]\n\nBiologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other scientists, 

In [18]:
#Serialization of Data (Python dictionary into JSON Formated String)
json.dumps(RESPONSE_JSON)
NUMBER=5 
SUBJECT="biology"
TONE="simple"

In [19]:
with get_openai_callback() as cb:
    resopnse = generate_evaluation_chain(
        {
            'text' : TEXT,'number' :NUMBER ,'subject' :SUBJECT,'tone' :TONE ,'response_json' :json.dumps(RESPONSE_JSON)
        }
    )

c:\Users\PRAMOD KHAVARE\MCQGEN\env\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each define

In [28]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}") #In dolller

Total Tokens:1474
Prompt Tokens:1067
Completion Tokens:407
Total Cost:0.0024145


In [39]:
quiz = (resopnse['quiz'])
quiz = json.loads(quiz)

In [45]:
quiz

{'1': {'mcq': 'Which of the following is a unifying theme in the field of biology?',
  'options': {'a': 'Evolution',
   'b': 'Chemistry',
   'c': 'Physics',
   'd': 'Geology'},
  'correct': 'a'},
 '2': {'mcq': 'What is the main function of cells in organisms?',
  'options': {'a': 'Processing energy',
   'b': 'Regulating internal environments',
   'c': 'Processing hereditary information',
   'd': 'Reproducing'},
  'correct': 'c'},
 '3': {'mcq': 'What is a key aspect of studying life in biology?',
  'options': {'a': 'Using the scientific method',
   'b': 'Studying only plants',
   'c': 'Ignoring evolution',
   'd': 'Not conducting experiments'},
  'correct': 'a'},
 '4': {'mcq': 'When did life on Earth emerge?',
  'options': {'a': '2.5 billion years ago',
   'b': '4.7 billion years ago',
   'c': '3.7 billion years ago',
   'd': '5.5 billion years ago'},
  'correct': 'c'},
 '5': {'mcq': 'What is the main goal of biologists studying biodiversity?',
  'options': {'a': 'Eliminating all organi

In [80]:
quiz_table_data = []
for keys,items in quiz.items():
    mcq = items['mcq']
    options = (items['options'])
    options = " | ".join([f"{key} : {value}" for key ,value in options.items()])
    correct = (items['correct'])
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})
print(quiz_table_data)

[{'MCQ': 'Which of the following is a unifying theme in the field of biology?', 'Choices': 'a : Evolution | b : Chemistry | c : Physics | d : Geology', 'Correct': 'a'}, {'MCQ': 'What is the main function of cells in organisms?', 'Choices': 'a : Processing energy | b : Regulating internal environments | c : Processing hereditary information | d : Reproducing', 'Correct': 'c'}, {'MCQ': 'What is a key aspect of studying life in biology?', 'Choices': 'a : Using the scientific method | b : Studying only plants | c : Ignoring evolution | d : Not conducting experiments', 'Correct': 'a'}, {'MCQ': 'When did life on Earth emerge?', 'Choices': 'a : 2.5 billion years ago | b : 4.7 billion years ago | c : 3.7 billion years ago | d : 5.5 billion years ago', 'Correct': 'c'}, {'MCQ': 'What is the main goal of biologists studying biodiversity?', 'Choices': 'a : Eliminating all organisms | b : Classifying and studying various forms of life | c : Ignoring the ecosystem | d : Focusing only on prokaryotic 

In [85]:
final_quiz = pd.DataFrame(
    quiz_table_data
)
final_quiz.to_csv('biology_quiz.csv' ,index=False)

In [92]:
abcc= [{"MCQ": {"A": 123 ,"B": 456}, "ANS": {1,2}}]
pd.DataFrame(abcc )

,MCQ,ANS
0,"{'A': 123, 'B': 456}","{1, 2}"
